In [1]:
from scipy.spatial import distance_matrix
from scipy.cluster.hierarchy import single,ward,complete,average,centroid,cophenet
from scipy.spatial.distance import pdist, squareform
import pandas as pd
import tqdm

In [2]:
from COS_Funcs.utils import read_data
from COS_Funcs.baseline.baseline import datasets

C:\Program Files (x86)\ANACONDA\lib\site-packages\_ctgan\synthesizer.py:7: TqdmExperimentalWarning: Using `tqdm.autonotebook.tqdm` in notebook mode. Use `tqdm.tqdm` instead to force console mode (e.g. in jupyter console)
  from tqdm.autonotebook import tqdm


In [11]:
# Not Normed
linkages = ['single','complete','average','centroid','ward']
df = pd.DataFrame(columns= linkages + ['best','worst'],index=datasets)

for dataset in tqdm.tqdm(datasets):
    X,y = read_data(dataset,norm=False)
#     X = X[y==1] # only min data
    cond_m = pdist(X)
    
    # linkages
    single_ = single(cond_m)
    complete_ = complete(cond_m)
    average_ = average(cond_m)
    centroid_ = centroid(cond_m)
    ward_ = ward(cond_m)
    
    # cophenetic correlation coefficient
    single_cp = cophenet(single_,cond_m)[0]
    complete_cp = cophenet(complete_,cond_m)[0]
    average_cp = cophenet(average_,cond_m)[0]
    centroid_cp = cophenet(centroid_,cond_m)[0]
    ward_cp = cophenet(ward_,cond_m)[0]
    
    # calculate best and worst
    coeffs = [single_cp,complete_cp,average_cp,centroid_cp,ward_cp]
    index_best = coeffs.index(max(coeffs))
    index_worst = coeffs.index(min(coeffs))
    b_w = [linkages[index_best],linkages[index_worst]]
    
    df.loc[dataset] = [single_cp,complete_cp,average_cp,centroid_cp,ward_cp]+b_w
    
df

100%|██████████| 75/75 [00:35<00:00,  2.13it/s]


,single,complete,average,centroid,ward,best,worst
Datasets\abalone11–17.csv,0.66954,0.569085,0.809794,0.796138,0.697115,average,complete
Datasets\abalone19.csv,0.71834,0.74582,0.825585,0.822684,0.803468,average,single
Datasets\abalone4–8.csv,0.794831,0.812274,0.836755,0.836575,0.817494,average,single
Datasets\abalone5–10.csv,0.693758,0.697285,0.816107,0.813358,0.788229,average,single
Datasets\abalone9-18.csv,0.73478,0.690038,0.826713,0.822252,0.776648,average,complete
...,...,...,...,...,...,...,...
Datasets\yeast-2_vs_8.csv,0.843291,0.566257,0.882764,0.892458,0.588895,centroid,complete
Datasets\yeast1.csv,0.640786,0.565544,0.764546,0.77636,0.506796,centroid,ward
Datasets\yeast3.csv,0.640786,0.565544,0.764546,0.775063,0.506796,centroid,ward
Datasets\yeast4.csv,0.640786,0.565544,0.764546,0.775037,0.506796,centroid,ward


In [12]:
df.to_csv('CPCC_no_std.csv')